# Interactive Ranking

In [2]:
%matplotlib tk

from sorbetto.core.entity import Entity
from sorbetto.performance.two_class_classification_performance import TwoClassClassificationPerformance
from sorbetto.ranking.ranking_induced_by_score import RankingInducedByScore
from sorbetto.ranking.ranking_score import RankingScore
from sorbetto.tile.tile import Tile
from sorbetto.parameterization import ParameterizationDefault

from random import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backend_bases import MouseButton

In [3]:
ca = Entity( TwoClassClassificationPerformance( random(), random(), random(), random() ), 'Classifier "A"' )
cb = Entity( TwoClassClassificationPerformance( random(), random(), random(), random() ), 'Classifier "B"' )
cc = Entity( TwoClassClassificationPerformance( random(), random(), random(), random() ), 'Classifier "C"' )
cd = Entity( TwoClassClassificationPerformance( random(), random(), random(), random() ), 'Classifier "D"' )
ce = Entity( TwoClassClassificationPerformance( random(), random(), random(), random() ), 'Classifier "E"' )

In [4]:
entities = [ ca, cb, cc, cd, ce ]
score = RankingScore.getF ( beta=1.0 )
ranking = RankingInducedByScore ( entities, score )

In [5]:
print ( ranking.values )

[0.20442814 0.68889051 0.2260841  0.73844343 0.72790254]


In [6]:
print ( ranking.getAllMinRanks() )

[5 3 4 1 2]


In [7]:
print ( ranking.getAllMaxRanks() )

[5 3 4 1 2]


In [8]:
for r in range ( len ( entities ) ) :
    print ( r+1, ':', (ranking.getEntitiesAtRank ( r+1 ) ) )

1 : [<sorbetto.core.entity.Entity object at 0x16d5222a0>]
2 : [<sorbetto.core.entity.Entity object at 0x16d522450>]
3 : [<sorbetto.core.entity.Entity object at 0x16d522360>]
4 : [<sorbetto.core.entity.Entity object at 0x16d522330>]
5 : [<sorbetto.core.entity.Entity object at 0x16d4e5b80>]


In [9]:

parameterization = ParameterizationDefault ()
min_x, max_x = parameterization.getBoundsParameter1 ()
min_y, max_y = parameterization.getBoundsParameter2 ()
extent = [ min_x, max_x, min_y, max_y ]


In [10]:
def draw_ladder(ax, coords):
    importance = parameterization.getCanonicalImportance(*coords)

    score = RankingScore(importance)
    ranking = RankingInducedByScore ( entities, score )

    names = np.asarray([ e.name for e in ranking.entities ])
    scores = ranking.values

    
    # Sort by scores (descending order for ranking)
    sorted_indices = np.argsort(scores)[::-1]
    sorted_scores = scores[sorted_indices]
    sorted_names = names[sorted_indices]
    
    # Normalize scores to y-positions
    if len(np.unique(sorted_scores)) > 1:
        y_positions = (sorted_scores - sorted_scores.min()) / (sorted_scores.max() - sorted_scores.min())
    else:
        y_positions = np.ones_like(sorted_scores) * 0.5
    
    # Scale to desired height
    y_positions = y_positions * 8 + 1
    
    # Draw main vertical line
    ax.plot([0, 0], [0, 10], 'k-', linewidth=2)
    
    # Group equal scores
    unique_scores = np.unique(sorted_scores)
    
    for score in unique_scores:
        mask = sorted_scores == score
        score_names = sorted_names[mask]
        score_y = y_positions[mask][0]  # All equal scores have same y position
        
        if len(score_names) == 1:
            # Single score - simple line and label
            ax.plot([0, 1], [score_y, score_y], 'k-', linewidth=1)
            ax.text(1.1, score_y, score_names[0], va='center', fontsize=10)
        else:
            # Multiple equal scores - draw bracket
            bracket_height = 0.3
            bracket_top = score_y + bracket_height/2
            bracket_bottom = score_y - bracket_height/2
            
            # Draw bracket
            ax.plot([0, 0.8], [score_y, score_y], 'k-', linewidth=1)  # horizontal to bracket
            ax.plot([0.8, 0.8], [bracket_bottom, bracket_top], 'k-', linewidth=1)  # vertical bracket
            ax.plot([0.8, 1], [bracket_top, bracket_top], 'k-', linewidth=1)  # top bracket line
            ax.plot([0.8, 1], [bracket_bottom, bracket_bottom], 'k-', linewidth=1)  # bottom bracket line
            
            # Add names
            for i, name in enumerate(score_names):
                name_y = bracket_top - (i + 0.5) * bracket_height / len(score_names)
                ax.text(1.1, name_y, name, va='center', fontsize=10)
        
        # Add score markers on main line
        ax.plot(0, score_y, 'ko', markersize=6)
    
    # Add score labels on the left
    for score, y_pos in zip(unique_scores, [y_positions[sorted_scores == score][0] for score in unique_scores]):
        ax.text(-0.2, y_pos, f'{score:.2f}', ha='right', va='center', fontsize=9)
    
    # Clean up the plot
    ax.set_xlim(-0.5, 3)
    ax.set_ylim(-0.5, 10.5)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title('Method Rankings', fontsize=12, pad=20)


In [11]:
tile = Tile ( name = "my tile", parameterization=parameterization )

fig, ax = plt.subplots (1, 2)
tile.draw ( fig, ax[0] )
# TODO: transform the following into an annotation
for i, entity_1 in enumerate ( entities ) :
    for j, entity_2 in enumerate ( entities ) :
        if i < j :
            print ( "plotting curve", i, j )
            curve = score.equivalent ( entity_1.performance, entity_2.performance )
            curve.draw (fig, ax[0], extent)


def on_click(event):
    if event.button is MouseButton.LEFT:
        ax[1].clear()
        draw_ladder(ax[1], (event.xdata, event.ydata))
        plt.show()

plt.connect('button_press_event', on_click)
plt.show()

plotting curve 0 1
plotting curve 0 2
plotting curve 0 3
plotting curve 0 4
plotting curve 1 2
plotting curve 1 3
plotting curve 1 4
plotting curve 2 3
plotting curve 2 4
plotting curve 3 4
